# Recurrent Neural Network

In [1]:
import os
import random
import numpy as np
import tensorflow as tf

# Set global seed
SEED = 42
os.environ['PYTHONHASHSEED'] = str(SEED)
random.seed(SEED)
np.random.seed(SEED)
tf.random.set_seed(SEED)


In [2]:
import numpy as np

data = np.load("C:/Users/prajw/Desktop/Undergrad Research/Datasets/rnn_dataset_with_subjects.npz")
X_rnn = data["X"]
y_labels = data["y"]
subject_ids = data["subject_ids"]

print("First subject ID:", subject_ids[0])
print("Shape of X_rnn:", X_rnn.shape)


First subject ID: 1018959
Shape of X_rnn: (620, 260, 190)


## Loading the phenotypic data to merge with ltsm 

In [7]:
import pandas as pd
FC_pheno_data = pd.read_csv("C:/Users/prajw/Desktop/Undergrad Research/Datasets/Preprocessed FC matrix with Pheno/FC_Merged.csv", index_col = 0)
FC_pheno_data.index.name = "Subject ID"

phenotype_cols = ['Inattentive', 'Hyper/Impulsive', 'Verbal IQ', 
                  'Performance IQ', 'Full4 IQ', 'Med Status', 'DX']
pheno_data = FC_pheno_data[phenotype_cols].copy()
pheno_data.index.name = 'Subject ID'
pheno_data['DX'] = pheno_data['DX'].apply(lambda x: 1 if x > 0 else 0)
pheno_data

,Inattentive,Hyper/Impulsive,Verbal IQ,Performance IQ,Full4 IQ,Med Status,DX
Subject ID,,,,,,,
1018959,47.0,44.0,99.0,115.0,103.0,1,0
1019436,60.0,66.0,124.0,108.0,122.0,1,1
1043241,40.0,43.0,128.0,106.0,120.0,1,0
1266183,44.0,43.0,136.0,96.0,120.0,1,0
1535233,41.0,43.0,106.0,135.0,122.0,1,0
...,...,...,...,...,...,...,...
5669389,15.0,9.0,120.0,97.0,110.0,1,0
6383713,29.0,32.0,115.0,91.0,104.0,1,1
6477085,13.0,12.0,115.0,112.0,115.0,1,0


In [11]:
# Make sure subject_ids and pheno_data index are both zero-padded strings
subject_ids = np.array([str(sid).zfill(7) for sid in subject_ids])
pheno_data.index = pheno_data.index.astype(str).str.zfill(7)

# Now match properly
matching_ids = [sid for sid in subject_ids if sid in pheno_data.index]
print(f"Matched subjects: {len(matching_ids)}")  # Should now be ~493

# Get indices of matching IDs from full list
matching_indices = [np.where(subject_ids == sid)[0][0] for sid in matching_ids]

# Subset data accordingly
X_rnn_matched = X_rnn[matching_indices]
y_labels_matched = y_labels[matching_indices]

phenotype_cols = ['Inattentive', 'Hyper/Impulsive', 'Verbal IQ', 
                  'Performance IQ', 'Full4 IQ', 'Med Status']
pheno_subset = pheno_data.loc[matching_ids, phenotype_cols].fillna(0)

from sklearn.preprocessing import StandardScaler
scaler = StandardScaler()
X_pheno_scaled = scaler.fit_transform(pheno_subset)

# Final checks
print("X_rnn_matched shape:", X_rnn_matched.shape)
print("X_pheno_scaled shape:", X_pheno_scaled.shape)
print("y_labels_matched shape:", y_labels_matched.shape)


Matched subjects: 493
X_rnn_matched shape: (493, 260, 190)
X_pheno_scaled shape: (493, 6)
y_labels_matched shape: (493,)


In [25]:
from sklearn.model_selection import train_test_split

X_ts_train, X_ts_test, X_pheno_train, X_pheno_test, y_train_all, y_test = train_test_split(
    X_rnn_matched, X_pheno_scaled, y_labels_matched,
    test_size=0.2,
    stratify=y_labels_matched,
    random_state=42
)

print("Train shape (fMRI):", X_ts_train.shape)
print("Test shape (fMRI):", X_ts_test.shape)
print("Train shape (pheno):", X_pheno_train.shape)
print("Test shape (pheno):", X_pheno_test.shape)
print("y_train_all shape:", y_train_all.shape)
print("y_test shape:", y_test.shape)


Train shape (fMRI): (394, 260, 190)
Test shape (fMRI): (99, 260, 190)
Train shape (pheno): (394, 6)
Test shape (pheno): (99, 6)
y_train_all shape: (394,)
y_test shape: (99,)


In [29]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, Dropout, concatenate, RepeatVector, Attention, Concatenate
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# Build early fusion LSTM model
def build_early_fusion_model(ts_shape=(260, 190), pheno_dim=6):
    input_ts = Input(shape=ts_shape)                            # (260, 190)
    input_pheno = Input(shape=(pheno_dim,))                     # (6,)

    x_ts = Masking(mask_value=0.0)(input_ts)

    # Repeat phenotype features to match time steps (260,)
    x_pheno = RepeatVector(ts_shape[0])(input_pheno)            # (260, 6)

    # Concatenate along last axis: (260, 190+6)
    fused_input = Concatenate(axis=-1)([x_ts, x_pheno])

    # LSTM layers
    x = LSTM(64, return_sequences=True)(fused_input)
    x = Dropout(0.3)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = Dropout(0.2)(x)

    # Attention mechanism
    attention_scores = Dense(1, activation='tanh')(x)
    attention_weights = tf.nn.softmax(attention_scores, axis=1)
    context_vector = tf.reduce_sum(attention_weights * x, axis=1)

    # Dense output layers
    x = Dense(32, activation='relu')(context_vector)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=[input_ts, input_pheno], outputs=output)
    model.compile(optimizer=RMSprop(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [35]:
from sklearn.utils import class_weight

# Class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_labels_matched), y=y_labels_matched)
class_weight_dict = dict(enumerate(class_weights))

# Step 2: Use only training data for CV
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
all_acc, all_prec, all_recall, all_f1 = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(y_train_all, y_train_all), start=1):
    print(f"\nFold {fold}")
    
    X_cv_train_ts, X_cv_val_ts = X_train_ts[train_idx], X_train_ts[val_idx]
    X_cv_train_pheno, X_cv_val_pheno = X_train_pheno[train_idx], X_train_pheno[val_idx]
    y_cv_train, y_cv_val = y_train_all[train_idx], y_train_all[val_idx]

    model = build_early_fusion_model()
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        [X_cv_train_ts, X_cv_train_pheno], y_cv_train,
        validation_data=([X_cv_val_ts, X_cv_val_pheno], y_cv_val),
        epochs=20,
        batch_size=32,
        callbacks=[early_stop],
        class_weight=class_weight_dict,
        verbose=0
    )

    y_pred_probs = model.predict([X_cv_val_ts, X_cv_val_pheno])
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    # Evaluation
    acc = accuracy_score(y_cv_val, y_pred)
    prec = precision_score(y_cv_val, y_pred)
    recall = recall_score(y_cv_val, y_pred)
    f1 = f1_score(y_cv_val, y_pred)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    all_acc.append(acc)
    all_prec.append(prec)
    all_recall.append(recall)
    all_f1.append(f1)


# Final summary
print("\nCross-Validation Summary:")
print(f"Avg Accuracy:  {np.mean(all_acc):.4f}")
print(f"Avg Precision: {np.mean(all_prec):.4f}")
print(f"Avg Recall:    {np.mean(all_recall):.4f}")
print(f"Avg F1-score:  {np.mean(all_f1):.4f}")




Fold 1
2/2 [==============================] - 3s 39ms/step
Accuracy: 0.5500, Precision: 0.5000, Recall: 0.3333, F1: 0.4000

Fold 2
2/2 [==============================] - 2s 42ms/step
Accuracy: 0.5750, Precision: 1.0000, Recall: 0.0556, F1: 0.1053

Fold 3
2/2 [==============================] - 2s 29ms/step
Accuracy: 0.5000, Precision: 0.4167, Recall: 0.2778, F1: 0.3333

Fold 4
2/2 [==============================] - 2s 42ms/step
Accuracy: 0.5000, Precision: 0.4583, Recall: 0.6111, F1: 0.5238

Fold 5
2/2 [==============================] - 2s 38ms/step
Accuracy: 0.5385, Precision: 0.4545, Recall: 0.2941, F1: 0.3571

Fold 6
2/2 [==============================] - 3s 37ms/step
Accuracy: 0.4872, Precision: 0.4516, Recall: 0.8235, F1: 0.5833

Fold 7
2/2 [==============================] - 3s 37ms/step
Accuracy: 0.5385, Precision: 0.3333, Recall: 0.0588, F1: 0.1000

Fold 8
2/2 [==============================] - 3s 40ms/step
Accuracy: 0.5385, Precision: 0.4783, Recall: 0.6471, F1: 0.5500

Fold 9


In [45]:
# Predict on aligned test set
y_test_probs = model.predict([X_ts_test, X_pheno_test])
y_test_pred = (y_test_probs > 0.5).astype(int).flatten()

# Evaluation
print("\nHold-Out Test Set Performance:")
print(classification_report(y_test, y_test_pred, target_names=["Control", "ADHD"]))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


4/4 [==============================] - 0s 45ms/step

Hold-Out Test Set Performance:
              precision    recall  f1-score   support

     Control       0.27      0.11      0.16        55
        ADHD       0.36      0.64      0.46        44

    accuracy                           0.34        99
   macro avg       0.32      0.37      0.31        99
weighted avg       0.31      0.34      0.29        99

Confusion Matrix:
[[ 6 49]
 [16 28]]


In [47]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, Masking, LSTM, Dense, Dropout
from tensorflow.keras.callbacks import EarlyStopping
from tensorflow.keras.optimizers import RMSprop
import numpy as np
import tensorflow as tf

# Reproducibility
SEED = 42
np.random.seed(SEED)
tf.random.set_seed(SEED)

# LSTM model using only time series data
def build_lstm_model(ts_shape=(260, 190)):
    input_ts = Input(shape=ts_shape)               # Only time series input

    x = Masking(mask_value=0.0)(input_ts)
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = Dropout(0.2)(x)

    # Attention mechanism
    attention_scores = Dense(1, activation='tanh')(x)
    attention_weights = tf.nn.softmax(attention_scores, axis=1)
    context_vector = tf.reduce_sum(attention_weights * x, axis=1)

    x = Dense(32, activation='relu')(context_vector)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_ts, outputs=output)
    model.compile(optimizer=RMSprop(1e-4), loss='binary_crossentropy', metrics=['accuracy'])

    return model


In [51]:
from sklearn.model_selection import train_test_split

# Use matched arrays with phenotype data (after alignment)
X_train_ts, X_test_ts, y_train_all, y_test = train_test_split(
    X_rnn_matched, y_labels_matched, test_size=0.2, stratify=y_labels_matched, random_state=42
)


In [53]:
from sklearn.utils import class_weight
from sklearn.model_selection import StratifiedKFold
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score

# Class weights
class_weights = class_weight.compute_class_weight('balanced', classes=np.unique(y_train_all), y=y_train_all)
class_weight_dict = dict(enumerate(class_weights))

# 10-fold CV
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=SEED)
all_acc, all_prec, all_recall, all_f1 = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train_ts, y_train_all), start=1):
    print(f"\nFold {fold}")
    X_cv_train_ts, X_cv_val_ts = X_train_ts[train_idx], X_train_ts[val_idx]
    y_cv_train, y_cv_val = y_train_all[train_idx], y_train_all[val_idx]

    model = build_lstm_model()
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_cv_train_ts, y_cv_train,
        validation_data=(X_cv_val_ts, y_cv_val),
        epochs=20,
        batch_size=32,
        callbacks=[early_stop],
        class_weight=class_weight_dict,
        verbose=0
    )

    y_pred_probs = model.predict(X_cv_val_ts)
    y_pred = (y_pred_probs > 0.5).astype(int).flatten()

    acc = accuracy_score(y_cv_val, y_pred)
    prec = precision_score(y_cv_val, y_pred)
    recall = recall_score(y_cv_val, y_pred)
    f1 = f1_score(y_cv_val, y_pred)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")

    all_acc.append(acc)
    all_prec.append(prec)
    all_recall.append(recall)
    all_f1.append(f1)

# Final summary
print("\nCross-Validation Summary:")
print(f"Avg Accuracy:  {np.mean(all_acc):.4f}")
print(f"Avg Precision: {np.mean(all_prec):.4f}")
print(f"Avg Recall:    {np.mean(all_recall):.4f}")
print(f"Avg F1-score:  {np.mean(all_f1):.4f}")



Fold 1
2/2 [==============================] - 2s 27ms/step
Accuracy: 0.6000, Precision: 0.5417, Recall: 0.7222, F1: 0.6190

Fold 2
2/2 [==============================] - 2s 41ms/step
Accuracy: 0.6000, Precision: 0.5556, Recall: 0.5556, F1: 0.5556

Fold 3
2/2 [==============================] - 2s 33ms/step
Accuracy: 0.6500, Precision: 0.5909, Recall: 0.7222, F1: 0.6500

Fold 4
2/2 [==============================] - 3s 49ms/step
Accuracy: 0.5500, Precision: 0.5000, Recall: 0.5556, F1: 0.5263

Fold 5
2/2 [==============================] - 2s 41ms/step
Accuracy: 0.5641, Precision: 0.5000, Recall: 0.4118, F1: 0.4516

Fold 6
2/2 [==============================] - 2s 36ms/step
Accuracy: 0.6410, Precision: 0.6000, Recall: 0.5294, F1: 0.5625

Fold 7
2/2 [==============================] - 2s 45ms/step
Accuracy: 0.6410, Precision: 0.5714, Recall: 0.7059, F1: 0.6316

Fold 8
2/2 [==============================] - 2s 34ms/step
Accuracy: 0.5897, Precision: 0.5333, Recall: 0.4706, F1: 0.5000

Fold 9


In [55]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on hold-out test set
y_test_probs = model.predict(X_test_ts)
y_test_pred = (y_test_probs > 0.5).astype(int).flatten()

# Evaluation
print("\nHold-Out Test Set Performance:")
print(classification_report(y_test, y_test_pred, target_names=["Control", "ADHD"]))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


4/4 [==============================] - 0s 51ms/step

Hold-Out Test Set Performance:
              precision    recall  f1-score   support

     Control       0.60      0.53      0.56        55
        ADHD       0.49      0.57      0.53        44

    accuracy                           0.55        99
   macro avg       0.55      0.55      0.54        99
weighted avg       0.55      0.55      0.55        99

Confusion Matrix:
[[29 26]
 [19 25]]


## Merging the pheno with time series acorss the time series

In [58]:
# Repeat phenotypic features for each time step (260) per subject
X_pheno_tiled = np.repeat(X_pheno_scaled[:, np.newaxis, :], repeats=260, axis=1)

print("X_pheno_tiled shape:", X_pheno_tiled.shape)  


X_pheno_tiled shape: (493, 260, 6)


In [60]:
# Concatenate along the last axis (feature dimension)
X_combined = np.concatenate([X_rnn_matched, X_pheno_tiled], axis=-1)

print("X_combined shape:", X_combined.shape) 


X_combined shape: (493, 260, 196)


## Splitting the data

In [63]:
from sklearn.model_selection import train_test_split

# 80/20 stratified split
X_train, X_test, y_train, y_test = train_test_split(
    X_combined, y_labels_matched,
    test_size=0.2,
    stratify=y_labels_matched,
    random_state=42
)

print(f"Train shape: {X_train.shape}")
print(f"Test shape:  {X_test.shape}")
print(f"y_train shape: {y_train.shape}, y_test shape: {y_test.shape}")


Train shape: (394, 260, 196)
Test shape:  (99, 260, 196)
y_train shape: (394,), y_test shape: (99,)


## Defining the model

In [66]:
from tensorflow.keras.models import Model
from tensorflow.keras.layers import Input, LSTM, Dense, Dropout, Masking
from tensorflow.keras.layers import Attention
from tensorflow.keras.optimizers import RMSprop
import tensorflow as tf

def build_fused_lstm_model(input_shape=(260, 196)):
    input_layer = Input(shape=input_shape)  # (time_steps=260, features=196)
    
    x = Masking(mask_value=0.0)(input_layer)
    x = LSTM(64, return_sequences=True)(x)
    x = Dropout(0.3)(x)
    x = LSTM(32, return_sequences=True)(x)
    x = Dropout(0.2)(x)
    
    # Attention
    attention_scores = Dense(1, activation='tanh')(x)
    attention_weights = tf.nn.softmax(attention_scores, axis=1)
    context_vector = tf.reduce_sum(attention_weights * x, axis=1)
    
    x = Dense(32, activation='relu')(context_vector)
    x = Dropout(0.2)(x)
    output = Dense(1, activation='sigmoid')(x)

    model = Model(inputs=input_layer, outputs=output)
    model.compile(optimizer=RMSprop(learning_rate=1e-4), loss='binary_crossentropy', metrics=['accuracy'])
    return model


## Cross Validation to train the model

In [69]:
from sklearn.model_selection import StratifiedKFold
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import accuracy_score, precision_score, recall_score, f1_score
from tensorflow.keras.callbacks import EarlyStopping

# Set up class weights
class_weights = compute_class_weight('balanced', classes=np.unique(y_train), y=y_train)
class_weight_dict = dict(enumerate(class_weights))

# Prepare Stratified K-Fold
kf = StratifiedKFold(n_splits=10, shuffle=True, random_state=42)
all_acc, all_prec, all_recall, all_f1 = [], [], [], []

for fold, (train_idx, val_idx) in enumerate(kf.split(X_train, y_train), 1):
    print(f"\nFold {fold}")

    X_fold_train, X_fold_val = X_train[train_idx], X_train[val_idx]
    y_fold_train, y_fold_val = y_train[train_idx], y_train[val_idx]

    model = build_fused_lstm_model()  # you already defined this
    early_stop = EarlyStopping(monitor='val_loss', patience=3, restore_best_weights=True)

    model.fit(
        X_fold_train, y_fold_train,
        validation_data=(X_fold_val, y_fold_val),
        epochs=20,
        batch_size=32,
        class_weight=class_weight_dict,
        callbacks=[early_stop],
        verbose=1
    )

    # Predictions
    y_pred = (model.predict(X_fold_val) > 0.5).astype(int).flatten()

    # Metrics
    acc = accuracy_score(y_fold_val, y_pred)
    prec = precision_score(y_fold_val, y_pred)
    recall = recall_score(y_fold_val, y_pred)
    f1 = f1_score(y_fold_val, y_pred)

    print(f"Accuracy: {acc:.4f}, Precision: {prec:.4f}, Recall: {recall:.4f}, F1: {f1:.4f}")
    
    all_acc.append(acc)
    all_prec.append(prec)
    all_recall.append(recall)
    all_f1.append(f1)

# Summary
print("\nCross-Validation Summary:")
print(f"Avg Accuracy:  {np.mean(all_acc):.4f}")
print(f"Avg Precision: {np.mean(all_prec):.4f}")
print(f"Avg Recall:    {np.mean(all_recall):.4f}")
print(f"Avg F1-score:  {np.mean(all_f1):.4f}")



Fold 1
Epoch 1/20
12/12 [==============================] - 12s 310ms/step - loss: 0.6875 - accuracy: 0.6158 - val_loss: 0.6812 - val_accuracy: 0.7000
Epoch 2/20
12/12 [==============================] - 1s 79ms/step - loss: 0.6750 - accuracy: 0.7316 - val_loss: 0.6718 - val_accuracy: 0.7750
Epoch 3/20
12/12 [==============================] - 1s 78ms/step - loss: 0.6677 - accuracy: 0.7260 - val_loss: 0.6626 - val_accuracy: 0.7750
Epoch 4/20
12/12 [==============================] - 1s 81ms/step - loss: 0.6583 - accuracy: 0.7655 - val_loss: 0.6558 - val_accuracy: 0.7750
Epoch 5/20
12/12 [==============================] - 1s 72ms/step - loss: 0.6512 - accuracy: 0.7599 - val_loss: 0.6483 - val_accuracy: 0.8000
Epoch 6/20
12/12 [==============================] - 1s 78ms/step - loss: 0.6460 - accuracy: 0.7740 - val_loss: 0.6407 - val_accuracy: 0.8250
Epoch 7/20
12/12 [==============================] - 1s 75ms/step - loss: 0.6326 - accuracy: 0.7825 - val_loss: 0.6302 - val_accuracy: 0.8250
Epo

## Hold out set testing

In [72]:
from sklearn.metrics import classification_report, confusion_matrix

# Predict on test set
y_test_probs = model.predict(X_test)
y_test_pred = (y_test_probs > 0.5).astype(int).flatten()

# Print report
print("\nHold-Out Test Set Performance:")
print(classification_report(y_test, y_test_pred, target_names=["Control", "ADHD"]))
print("Confusion Matrix:")
print(confusion_matrix(y_test, y_test_pred))


4/4 [==============================] - 0s 52ms/step

Hold-Out Test Set Performance:
              precision    recall  f1-score   support

     Control       0.81      0.85      0.83        55
        ADHD       0.80      0.75      0.78        44

    accuracy                           0.81        99
   macro avg       0.81      0.80      0.80        99
weighted avg       0.81      0.81      0.81        99

Confusion Matrix:
[[47  8]
 [11 33]]
